In [1]:
import networkx as nx
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
import random

In [2]:
G = nx.read_gml('GraphMissingEdges.gml')

edges_to_evaluate = pd.read_csv('edgesToEvaluate.csv')

edges_to_evaluate

linkID                  venue1                  venue2
0         9  mJ_ucQ2_3hfTsmCcKb-hgw  qXGKYRwCR9SLgLl0g_9o5g
1       135  y19xFolCozaRA-gGmHwkQA  F6c3D1o9Z4Tl6cDorb3WgA
2       434  R1GwW4C1gh2Nmue9K0WYVA  Ul6JwluSTm12PVDIqnNaTg
3       262  zzBa0pQjM1gov00bXjYYXg  3D6Uck9QSdxZKFstf5DGlg
4       383  U2d-meX4sVq0kiqcrpHt1w  vuDL_d3GYAtbvX9EJQqVog
..      ...                     ...                     ...
495     225  aSb4vkaMh7K2lHhnV2UIag  bQ-sXUqPSr4-iJfB764Nzw
496     288  6p39JCOx1L054G9jM10-5g  xwEYTGJ_82ScbpXcheqqQw
497     348  6WItftahZ9lNFJxfDPSJ0Q  XPmZnhnx0YeN8Xvo7y2xsA
498     187  GUriQoD_GHo6DNJlR1_CrA  1w6_xrdhVD-y-DBYpv0YCQ
499     368  CTiWs36A_sX3eahdqrYxUQ  X8wK-tnwx9ATKhQ7q39ILw

[500 rows x 3 columns]

In [3]:
indexes = []
data = []
for key, attrs in G.nodes.data():
    indexes.append(key)
    data.append(attrs)

nodes = pd.DataFrame(data, index=indexes)
nodes.head(10)

longitude   latitude                categories stars  \
ql0AaBp68ckekxvWOF8xLA -79.992016  40.438701                     1,362   3.0   
WHxonk9W_sRLk8cwOoZQqQ -80.064789  40.436045                   280,566   4.0   
P6HDtlj1GSu9UG2Aal2PPg -79.979840  40.386165        327,559,107,63,213   3.0   
3kUqNxO1rkDDb89GAfyNgw -79.925404  40.457973   338,280,247,292,671,546   4.5   
v_pED2nMFPsBGD4Tq2ygBw -80.001983  40.438355   407,247,270,645,438,488   2.0   
nZDIrGshkfLZf6ImQtAasQ -80.044442  40.382049      58,1,341,275,604,248   3.5   
cqPrr2uDMLP9_cPbIiOSrA -79.930231  40.456795             636,1,849,641   2.5   
O1ird5yRyuDFnOmYu90OoA -79.964218  40.466663  124,1,208,651,58,275,738   3.5   
-UjCvAsvBOr19y8lqPueiQ -79.917630  40.458928                     791,1   2.5   
zzBa0pQjM1gov00bXjYYXg -79.981787  40.429373           706,641,133,344   3.5   

                                                 name reviewCount  
ql0AaBp68ckekxvWOF8xLA                 Cafe Fifth Ave          20  
WHxonk9W_sRLk8cwOoZQqQ        Good Fellas Barber Shop          12  
P6HDtlj1GSu9UG2Aal2PPg     Tightspot Dancewear Center           4  
3kUqNxO1rkDDb89GAfyNgw  Evolve Wellness Spa Shadyside          95  
v_pED2nMFPsBGD4Tq2ygBw         Nova Dental Associates           5  
nZDIrGshkfLZf6ImQtAasQ       The Saloon of Mt Lebanon          60  
cqPrr2uDMLP9_cPbIiOSrA                  Bagel Factory          42  
O1ird5yRyuDFnOmYu90OoA           Round Corner Cantina         353  
-UjCvAsvBOr19y8lqPueiQ                   China Garden          22  
zzBa0pQjM1gov00bXjYYXg   South Side BBQ Company Truck          32

In [4]:
categories = pd.read_csv('categories.csv', index_col='CategoryId')

# First strategy

Use a classifier based on decision trees with gradient boosting (CatBoost), using node features, edge weight and shallow embeddings with node2vec.

## Training node2vec

In [8]:
random.seed(42)
# Remove training edges
prop = 0.25
edge_subset = random.sample(list(G.edges.data()), int(prop * G.number_of_edges()))

# Copy the graph without the training edges
G_edges_removed = G.copy()
G_edges_removed.remove_edges_from(edge_subset)

In [9]:
from node2vec import Node2Vec

node2vec = Node2Vec(G_edges_removed, dimensions=128, walk_length=80, num_walks=10, workers=8)

n2v_model = node2vec.fit(window=10, min_count=1)

Computing transition probabilities:   0%|          | 0/4575 [00:00<?, ?it/s]

Generating walks (CPU: 8): 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


## Generate dataset

In [10]:
# Negative sampling
negative_samples = list(nx.non_edges(G))
negative_samples = [(u, v, 1) for u, v in negative_samples]

# Positive sampling
positive_samples = edge_subset
positive_samples = [(u, v, p['weight']) for u, v, p in positive_samples]

In [11]:
NEGATIVES_SAMPLES_AMOUNT = 1
random.seed(42)
sampled_negative_samples = random.sample(negative_samples, NEGATIVES_SAMPLES_AMOUNT * len(positive_samples))

# remove edge from validation if they are on the testing set
def filter_evaluation_edges(sample):
    u, v, _ = sample
    a1 = edges_to_evaluate[(edges_to_evaluate.venue1 == u) & (edges_to_evaluate.venue2 == v)]
    a2 = edges_to_evaluate[(edges_to_evaluate.venue1 == v) & (edges_to_evaluate.venue2 == u)]
    return a1.empty and a2.empty

print(len(sampled_negative_samples))
sampled_negative_samples = list(filter(filter_evaluation_edges, tqdm(sampled_negative_samples)))
print(len(sampled_negative_samples))

4747


  0%|          | 0/4747 [00:00<?, ?it/s]

4747


### Preprocessing and feature engineering

The features chosen were:
- The Euclidean distance of the nodes
- MSE of the shallow embeddings
- The categories, coded in a bag of words style, with a feature for each category, with 0 if there is no category and 1 if there is a category. The categories of each node are added together in a single "bag of categories" for the edge.
- The number of categories that the bag of categories has more than 2, i.e. the size of the intersection of the set of categories for each node.
- The sum of the reviews for the nodes
- The number of average stars of the nodes

In [12]:
# Create dataset and labels
data = positive_samples + sampled_negative_samples
labels = np.array([1] * len(positive_samples) + [0] * len(sampled_negative_samples))

# bag of categories
def bag_of_categories(node_categories):
    cats = node_categories.split(',')
    boc = np.zeros(len(categories), dtype=np.uint8)
    for cat in cats:
        try:
            boc[int(cat)] = 1
        except:
            pass
    return boc


def euclidean_dist(node1, node2):
    return math.sqrt((node2.latitude - node1.latitude)**2 + (node2.longitude - node1.longitude)**2)

def generate_data(data):
    for u, v, weight in tqdm(data, total=len(data)):
        edge_embedding = (n2v_model.wv[u] - n2v_model.wv[v]) ** 2 # mse of the shallow embeddings
        u_node = nodes.loc[u]
        v_node = nodes.loc[v]
        u_boc = bag_of_categories(u_node.categories) # BoC of u node
        v_boc = bag_of_categories(v_node.categories) # BoC of v node
        boc = u_boc + v_boc  # shared BoC
        dist = euclidean_dist(u_node, v_node) # euclidean distance using lat on long
        
        row = np.concatenate([
            edge_embedding,
            boc,
            np.array([
                (boc > 1).sum(), # boc intersection
                dist,
                (float(u_node.stars) + float(v_node.stars)) / 2, # average score
                int(u_node.reviewCount) + int(v_node.reviewCount), # review sum

            ]),
        ])
        yield row

generated_data = tuple(generate_data(data))
processed_data = np.vstack(generated_data)

  0%|          | 0/9494 [00:00<?, ?it/s]

## Classifier Training

The CatBoost classifier, a gradient boost algorithm, was chosen.

In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.ensemble import RandomForestClassifier 
from sklearn.pipeline import Pipeline
import pprint
from catboost import CatBoostClassifier

def create_model():
    model = CatBoostClassifier(iterations=100,
                               max_depth=10,
                               random_state=42,
                               verbose=False)
    return model

## Training

The k-fold cross validation was performed to validate the quality of the model with the training and test variation.

In [14]:
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []

def log_metrics(metrics):
    print('Metrics:')
    pprint.pprint(metrics, indent=4, width=40)

def compute_metrics(y_test, y_pred):
    return {
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'roc_auc_score': roc_auc_score(y_test, y_pred),
    }

for train_index, test_index in tqdm(kf.split(processed_data), total=k):
    X_train, X_test = processed_data[train_index], processed_data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    model = create_model()

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print('Finished... computing metrics')

    metrics = compute_metrics(y_test, y_pred)
    log_metrics(metrics)

  0%|          | 0/5 [00:00<?, ?it/s]

Finished... computing metrics
Metrics:
{   'accuracy': 0.8056872037914692,
    'f1': 0.8031999999999999,
    'precision': 0.8053475935828877,
    'recall': 0.801063829787234,
    'roc_auc_score': 0.805641403944712}
Finished... computing metrics
Metrics:
{   'accuracy': 0.8077935755660874,
    'f1': 0.8107827890098498,
    'precision': 0.8292682926829268,
    'recall': 0.7931034482758621,
    'roc_auc_score': 0.808380858858632}
Finished... computing metrics
Metrics:
{   'accuracy': 0.8251711427066877,
    'f1': 0.8226495726495726,
    'precision': 0.8351409978308026,
    'recall': 0.8105263157894737,
    'roc_auc_score': 0.8251788586323553}
Finished... computing metrics
Metrics:
{   'accuracy': 0.8056872037914692,
    'f1': 0.8027792624265099,
    'precision': 0.7896950578338591,
    'recall': 0.816304347826087,
    'roc_auc_score': 0.8060071279477727}
Finished... computing metrics
Metrics:
{   'accuracy': 0.8166491043203372,
    'f1': 0.8146964856230032,
    'precision': 0.825242718446

Once validated, the final training run is carried out, with all the training data available.

In [15]:
model = create_model()

model.fit(processed_data, labels)
print(np.argmax(model.get_feature_importance()), np.max(model.get_feature_importance()))
model.get_feature_importance()[-10:]

1024 9.89248568064104


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 3.3502119 , 5.66502338, 1.79620469, 9.89248568])

Prediction for false positive count in 5000 negative samples.

In [16]:
print('false positive rate:')
n_fp = model.predict(np.vstack(tuple(generate_data(negative_samples[:5000])))).sum()
n_fp, n_fp / 5000

false positive rate:


  0%|          | 0/5000 [00:00<?, ?it/s]

(748, 0.1496)

## Inference

In [17]:
edges_to_eval_list = [(u, v, 1) for _, _, u, v in edges_to_evaluate.to_records()]
final_data = np.vstack(tuple(generate_data(edges_to_eval_list)))
preds = model.predict(final_data)
out = pd.concat([edges_to_evaluate, pd.Series(preds, name='link')], axis=1)
out[['linkID', 'link']].to_csv('results_cat_boost.csv', columns=['linkID', 'link'],index=False)
preds.sum()

  0%|          | 0/500 [00:00<?, ?it/s]

246

## Results obtained on kaggle

- 0.81794 -> best tested result
- 0.80586 -> second best tested result